In [1]:
import os

# Your API key and project must be provided. Use the environment variables already
# wired up, or overwrite the following lines with your own values manually.
API_KEY = os.environ.get('SUST_API_KEY')
PROJECT = os.environ.get('SUST_PROJECT', 'cds-demo')

# This value will be provided to you for use during the workshop.
GOOGLE_MAPS_API_KEY = os.environ.get('GOOGLE_MAPS_API_KEY')

# Define your search input below.
SEARCH_ADDRESS = "595 Pacific Ave, San Francisco, CA"

In [2]:
from dataclasses import asdict, dataclass
from typing import List
import urllib.parse

import googlemaps
import pandas as pd
import requests


@dataclass
class Point:
    lat: float
    lng: float
    

def geocode(address: str) -> Point:
    cl = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)
    res = cl.geocode(address)

    if len(res) != 1:
        raise Exception(f"geocoder returned {len(res)} results, expected 1")

    loc = res[0]['geometry']['location']
    return Point(**loc)


def get_physical_risk_exposure(point: Point) -> dict:
    qp = urllib.parse.urlencode({
        "api_key": API_KEY,
        "project": PROJECT,
        "lat": point.lat,
        "lng": point.lng,
    })

    ep = f"https://explorer.sustglobal.io/api/experimental/search?{qp}"

    resp = requests.get(ep)
    if resp.status_code != 200:
        raise Exception(f"API request failed: code={resp.status_code} body={resp.text}")

    return resp.json()['features'][0]['properties']['physical_risk_exposure']

In [3]:
pt = geocode(SEARCH_ADDRESS)
print(f"Successfully resolved address: search=\"{SEARCH_ADDRESS}\" geom={pt}")

Successfully resolved address: search="595 Pacific Ave, San Francisco, CA" geom=Point(lat=37.7970119, lng=-122.4050949)


In [4]:
exp = get_physical_risk_exposure(pt)
print("Successfully retrieved risk summary data")

Successfully retrieved risk summary data


In [5]:
print("Local physical risk exposure")
pd.DataFrame(exp['local']['30yr']).sort_values(by=['hazard'])

Local physical risk exposure


,hazard,score,label
3,cyclone,0.000000,LOW
5,flood_potential,0.000000,LOW
0,heatwave,0.162500,MEDIUM
2,sea_level_rise,0.229840,MEDIUM
1,water_stress,0.463986,MEDIUM
4,wildfire,0.000000,LOW


In [6]:
print("Background physical risk exposure")
pd.DataFrame(exp['background']['30yr']).sort_values(by=['hazard'])

Background physical risk exposure


,hazard,score,label
0,cyclone,0.000000,LOW
3,flood_potential,0.000000,LOW
4,heatwave,0.160052,MEDIUM
5,sea_level_rise,0.229840,MEDIUM
2,water_stress,0.484284,MEDIUM
1,wildfire,0.000279,LOW
